In [2]:
import asyncio
import os

In [7]:
os.environ["PYTHONASYNCIODEBUG"]='YES'

### Try asyncio.run()

#### Code is from here :
https://realpython.com/async-io-python/

In [21]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
asyncio.run(main())
elapsed = time.perf_counter() - s
print(f" executed in {elapsed:0.2f} seconds.")

RuntimeError: asyncio.run() cannot be called from a running event loop

#### There is an event loop running already! Try to get the loop .

In [23]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
elapsed = time.perf_counter() - s
print(f" executed in {elapsed:0.2f} seconds.")

RuntimeError: This event loop is already running

### Aah! How can you run a a running loop?

### Try running main() using await

In [18]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
await main()
elapsed = time.perf_counter() - s
print(f" executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
 executed in 0.99 seconds.


### Now try loop.create_task

In [14]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
loop.create_task(main())
elapsed = time.perf_counter() - s
print(f" executed in {elapsed:0.2f} seconds.")

 executed in 0.00 seconds.
One
One
One
Two
Two
Two


### Notice that create task has run the main method in the BACKGROUND unlike ```await main()```

In [32]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
t = loop.create_task(main())
print(f"Type of t : {type(t)}")
await t
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

Type of t : <class '_asyncio.Task'>
One
One
One
Two
Two
Two
executed in 1.02 seconds.


## What does asyncio.gather do?

"While it doesn’t do anything tremendously special, gather() is meant to neatly put a collection of coroutines (futures) into a single future. As a result, it returns a single future object, and, if you await asyncio.gather() and specify multiple tasks or coroutines, you’re waiting for all of them to be completed."

So this should work the same:

In [39]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    g = asyncio.gather(count(), count(), count())
    print(f"Type of g : {type(g)}")
    await(g)

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
t = loop.create_task(main())
print(f"Type of t : {type(t)}")
await t
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

Type of t : <class '_asyncio.Task'>
Type of g : <class 'asyncio.tasks._GatheringFuture'>
One
One
One
Two
Two
Two
executed in 1.01 seconds.


And so should this :

In [35]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    g = asyncio.gather(count(), count(), count())
    print(f"Type of g : {type(g)}")
    await(g)

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
f = asyncio.gather(main())
print(f"Type of f : {type(f)}")
await f
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

Type of f : <class 'asyncio.tasks._GatheringFuture'>
Type of g : <class 'asyncio.tasks._GatheringFuture'>
One
One
One
Two
Two
Two
executed in 1.02 seconds.


In [ ]:
And so should this :

In [41]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    g = asyncio.gather(count(), count(), count())
    print(f"Type of g : {type(g)}")
    #await(g)

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
f = asyncio.gather(main())
#await f
print(f"Type of f : {type(f)}")
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

Type of f : <class 'asyncio.tasks._GatheringFuture'>
executed in 0.00 seconds.
Type of g : <class 'asyncio.tasks._GatheringFuture'>
One
One
One
Two
Two
Two


Seems like asyncio.gather also runs your coroutines in the background. You don't have to await it?

In [ ]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

import time
s = time.perf_counter()
loop = asyncio.get_event_loop()
await main()
elapsed = time.perf_counter() - s
print(f" executed in {elapsed:0.2f} seconds.")

### Read this at this point 

https://hynek.me/articles/waiting-in-asyncio/

https://stackoverflow.com/questions/55590343/asyncio-run-or-run-until-complete

### Also this

https://bbc.github.io/cloudfit-public-docs/asyncio/asyncio-part-2

#### Manual event loop interaction

If you’re using Python 3.6, and you need to run coroutines from ordinary sync code (which you probably will, if you want to start something.) then you will need to start the event loop. There are two methods for doing this:

asyncio.get_event_loop().run_forever()
will cause the event loop to run forever (or until explicitly killed). This isn’t usually particularly useful. Much more useful is:

r = asyncio.get_event_loop().run_until_complete(f)
which takes a single parameter. If the parameter is a future (such as a task) then the loop will be run until the future is done, returning its result or raising its exception. So putting it together:

async def example_coroutine_function():
    ...

loop = asyncio.get_event_loop()
t = loop.create_task(example_coroutine_function())
r = loop.run_until_complete(t)
will create a new task which executes example_coroutine_function inside the event loop until it finishes, and then return the result.

In fact this can be simplified further since if you pass a coroutine object as the parameter to run_until_complete then it automatically calls create_task for you.



### Also this
https://stackoverflow.com/questions/64851715/python-asyncio-future-vs-task#:~:text=In%20short%2C%20future%20is%20the,in%20fact%20strongly%20single%2Dthreaded.

Also, be careful not to confuse asyncio futures with concurrent.futures futures, the latter being indeed multi-threaded. Despite similarities in the API coming from the fact that asyncio futures were inspired by the ones from concurrent.futures, they work in conceptually different ways and cannot be used interchangeably.


An ordinary future is useful to connect callbacks, like JS promises. In particular it serves as bridge between callbacks and coroutines. You can create a future and pass it to a callback which will later invoke fut.set_result(x). A coroutine can await it in the meantime (or await several such futures with gather() etc.) When the callback completes the future, the coroutine that awaited it gets resumed and await <future> gets evaluated to the x the callback passed to set_result(). Asyncio plumbing code, the transports and protocols layer, is all written in this style.